In [5]:
import numpy
import pandas as pd


# import tsv file
alldata=pd.read_csv('inclass_ex.txt',sep='\t', header=1)

# seperate and label columns
labels=alldata[['#OTU ID','taxonomy']]
controls=alldata[['pcrblank','negcontrol1']]
data=alldata.drop(['#OTU ID','taxonomy','pcrblank','negcontrol1'],axis=1)


#function to remove contamination threshold
def decontaminate_column(data_column,control_column):
    column = data_column.copy()
    column.ix[column <= control_column] = 0
    return column

#function to iterate decontaminate_column across all samples
def decontaminate(dataframe,control):
    for control_name in control.columns.values.tolist():
        dataframe=dataframe.apply(lambda c: decontaminate_column(c, control[control_name]), axis=0)
    return dataframe
data=decontaminate(data,controls)

# Output cleaned BIOM table
cleaned_otu_table = pd.concat([labels['#OTU ID'],controls,data,labels['taxonomy']],axis=1)
with open('otu_table_mc2_w_tax__Project_Moorea__renamed_text.txt', 'r') as f:
    comment=f.readline()
with open('cleaned_otu_table.csv', 'w') as f:
    f.write(comment)
cleaned_otu_table.to_csv('cleaned_otu_table.csv', sep='\t', mode='a',index=False)